In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import timeit
import statistics
import time
import torch
import torch_geometric
import importlib

from data_utils import synthetic_data
from data_utils import graph_constructors
from data_utils import group_to_image_constructors

import networkx as nx

import importlib

from Unet import helper
from Unet import simulation

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

In [2]:
# from collections import defaultdict
# import torch.nn.functional as F
# from Unet.loss import dice_loss

# def calc_loss(pred, target, metrics, bce_weight=0.5):
#     bce = F.binary_cross_entropy_with_logits(pred, target)

#     pred = F.sigmoid(pred)
#     dice = dice_loss(pred, target)

#     loss = bce * bce_weight + dice * (1 - bce_weight)

#     metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
#     metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
#     metrics['loss'] += loss.data.cpu().numpy() * target.size(0)

In [5]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target)
    
    loss = (1 - ((2. * intersection + smooth) / (pred + target + smooth)))
    
    return loss.mean()

In [8]:
pred = torch.tensor([0.5, 0.5])
target = torch.tensor([1,0])

dice_loss(pred, target)

tensor(0.2667)

In [16]:
target = torch.tensor(np.random.randint(0,2, size = (4,2,224,224)))

In [17]:
pred = torch.tensor(np.random.uniform(size = (4,2,224,224)))

In [18]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    print('pred.shape: ', pred.shape, ', target.shape: ', target.shape)

    intersection = (pred * target).sum(dim=2).sum(dim=2)

    print('intersection.shape: ', intersection.shape)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()

In [19]:
dice_loss(pred, target, smooth = 1.)

pred.shape:  torch.Size([4, 2, 224, 224]) , target.shape:  torch.Size([4, 2, 224, 224])
intersection.shape:  torch.Size([4, 2])


tensor(0.4999, dtype=torch.float64)